## Data preprocessing
1. Import the observations built from single-cell data.
2. Standardize the observations gene names.
3. Get interaction graph from DoRothEA database.
4. Standardize the interaction graph gene names.
5. Eliminate genes with both no influence on others into the interaction graph and no expression value into the single-cell data.

## BoNesis selection of components

### 1. Import the observations built from single-cell data:

In [1]:
from typing import Dict
def get_dict_of_observations_from_file(path_data: str, sep="\t") -> Dict:
    """
    file format input:
     - 1st line: observed components names
     - 1st column: observations identifiers
    """
    observations_dict = dict()
    with open(path_data, 'r') as observations_file:
        for i, line in enumerate(observations_file):
            if i == 0:
                observed_components = line.replace('"','').strip("\n").split(sep)
            elif len(line) < 2:
                continue
            else:
                observation = line.replace('"','').strip().split(sep)
                observations_dict[observation[0]] = dict()
                for j, component in enumerate(observed_components):
                    if j == 0:
                        continue #First column contains the observation identifier.
                    if "NA" not in observation[j]:
                        observations_dict[observation[0]][component] = int(observation[j])
    return observations_dict

In [2]:
# Save in a dictionary the observations stored in a file:
observations_from_singlecell_nestorowa = get_dict_of_observations_from_file("nestorowa_binarizedObservations.csv", sep=",")

# Visualisation:
import pandas as pd
pd.DataFrame.from_dict(observations_from_singlecell_nestorowa, orient="index").fillna('')

,1110008L16Rik,1110059E24Rik,1600014C10Rik,1600020E01Rik,1700006J14Rik,1700017B05Rik,1700030K09Rik,2010315B03Rik,2210016F16Rik,2210016L21Rik,...,Slc25a3,Snrpb,Sod1,Strap,Tspan8,Tubb4b,Txndc16,Ucp2,Uqcrc1,Vwf
S1,0,0,0,0,0,0,0,0,0,0,...,,,,,,,,,,
S4,0,1,1,0,0,0,0,0,1,0,...,,,,,,,,,,
S0,0,1,0,0,0,0,0,0,0,0,...,,,,,,,,,,
S2,0,0,1,1,0,0,0,0,0,0,...,,,,,,,,,,
S5,0,0,0,0,1,0,0,0,0,0,...,,,,,,,,,,
S3,0,0,0,0,0,1,0,0,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0


### 2. Standardize the observations gene names.

In [3]:
import os
from typing import List, Set, Dict, Tuple


def get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata: str) -> Dict:
    """
    Create a dictionary matching each possible gene name to its NCBI symbol.
    
    Particularity:
    Creation of a temporary file for speeding up the task facing a large matrix from NCBI, the parsing of the NCBI gene data is run with awk. A temporary file is then created.
    
    INPUT
        path_NCBIgenedata: path to the NCBI gene data
    OUTPUT
        dictionary (key: gene name, value: reference gene name (being the NCBI symbol))
    """
    
    # Parse the downloaded NCBI gene data:
    path_NCBIgenedata_cut = f"{path_NCBIgenedata}_cut"
    command_parsing = "awk -F'\t' '{print $3 \"\t\" $5 \"\t\" $11}' " + path_NCBIgenedata + " | tr \| '\t' > " + path_NCBIgenedata_cut + " ; sed -i 1d " + path_NCBIgenedata_cut
    os.system(command_parsing)
    
    # Extract gene data information:    
    gene_synonyms_dict = dict()
    symbols = set()

    with open (path_NCBIgenedata_cut, "r") as file_synonyms:
        for gene in file_synonyms:
            gene = gene.strip().upper()
            gene_symbols_list = gene.split("\t")
            #extract reference gene symbol:
            ncbi_symbol = gene_symbols_list.pop(0)
            #delete non-informative synonyms:
            res = [syn for syn in gene_symbols_list if (syn != "-" and syn != ncbi_symbol)]

            #create the dictionnary matching each symbol to its reference gene symbol (NCBI symbol):
            gene_synonyms_dict[ncbi_symbol] = ncbi_symbol
            symbols.add(ncbi_symbol)

            for gene in res:
                if gene not in symbols:
                    # Warning with NCBI list of synonyms:
                    # A noun can be the synonym of several symbols.
                    # Arbitrary, the choosen one is the first.
                    gene_synonyms_dict[gene] = ncbi_symbol
                    
    os.system(f"rm {path_NCBIgenedata_cut}")
    return gene_synonyms_dict


def get_reference_gene_name(gene_name: str, dict_synonyms: dict) -> str:
    """
    Given a gene name, return its reference name.
    INPUT
        dict_synonyms
        gene_name: the gene name you want its reference name
    OUTPUT
        the synonym considered as the reference name
    """
    gene_name = gene_name.upper()
    if gene_name in dict_synonyms:
        return dict_synonyms[gene_name]
    return gene_name

In [4]:
def standardize_genename_in_dict_of_observations(observations_dict: Dict, path_NCBIgenedata: str) -> Dict:
    """
    Create a copy of the input dict of observations, with each gene name replaced by its reference name (NCBI symbol).
    
    Require the following functions:
        get_dict_matching_synonyms_to_refgenename
        get_reference_gene_name
        
    INPUT
        observations_dict: dict (key = observation identifier, value = dict (key = genename, value = gene status))
        path_NCBIgenedata: path to the NCBI gene data
    OUTPUT
        dict (key = observation identifier, value = dict (key = reference genename, value = gene status))
    """
    
    # Get gene data information:
    gene_synonyms_dict = get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata)
    
    # Copy the dict of observations by replacing each genename by its reference genename (NCBI symbol) into it:
    standardized_observations_dict = dict()
    
    for k,v in observations_dict.items():
        standardized_observations_dict[k] = dict()
        for component, status in v.items():
            standardized_component = get_reference_gene_name(component, gene_synonyms_dict)
            standardized_observations_dict[k][standardized_component] = status
    
    return standardized_observations_dict

In [5]:
# Copy the dict of observations by replacing each gene name by its NCBI symbol:
standardized_observations_from_singlecell_nestorowa = standardize_genename_in_dict_of_observations(observations_from_singlecell_nestorowa, "Mus_musculus.gene_info.20221005.tsv")

In [6]:
# Visualisation of the matrix with standardized gene names:
df = pd.DataFrame.from_dict(standardized_observations_from_singlecell_nestorowa, orient="index").fillna('')
df

,PRORP,1110059E24RIK,1600014C10RIK,1600020E01RIK,1700006J14RIK,1700017B05RIK,1700030K09RIK,2010315B03RIK,2210016F16RIK,2210016L21RIK,...,SLC25A3,SNRPB,SOD1,STRAP,TSPAN8,TUBB4B,TXNDC16,UCP2,UQCRC1,VWF
S1,0,0,0,0,0,0,0,0,0,0,...,,,,,,,,,,
S4,0,1,1,0,0,0,0,0,1,0,...,,,,,,,,,,
S0,0,1,0,0,0,0,0,0,0,0,...,,,,,,,,,,
S2,0,0,1,1,0,0,0,0,0,0,...,,,,,,,,,,
S5,0,0,0,0,1,0,0,0,0,0,...,,,,,,,,,,
S3,0,0,0,0,0,1,0,0,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0


In [7]:
# Set of the genes with expression values in this single-cell dataset:
standardized_genenames_in_singlecell = set(df)

### 3. Get interaction graph from DoRothEA database.

In [8]:
import os
import datetime

def dorothea_extraction(organism: str="human", confidence: str="AB", directory_output: str="./"):
    ''' Store in a SIF file the subnetwork from DoRothEA database about mus musculus given the confidence on edges.
    INPUT
        organism: string that can be human or mouse
        confidence: string in the set A, AB (default), ABC, ABCD, ABCDE
        output directory: the current one by default
    OUTPUT
        SIF file in the directory_output, under the format "YYYY-MM-DD_dorotheaX.sif" with X the confidence given in argument
    '''
    
    assert organism == 'human' or organism == 'mouse', f"organism must be human or mouse"
    
    date = datetime.datetime.now()
    
    import rpy2.robjects as robjects
    import rpy2.robjects.packages as rpackages

    if confidence == "A":
        confidenceR = '"A"'
    elif confidence == "AB":
        confidenceR = '"A","B"'
    elif confidence == "ABC":
        confidenceR = '"A","B","C"'
    elif confidence == "ABCD":
        confidenceR = '"A","B","C","D"'
    elif confidence == "ABCDE":
        confidenceR = '"A","B","C","D","E"'
    else:
        raise InputError("Incorrect argument: confidence for edges can be A, AB, ABC, ABCD, ABCDE")

    #robjects.r('''
    #    if (!requireNamespace("BiocManager", quietly = TRUE))
    #        install.packages("BiocManager")
    #    BiocManager::install("dorothea")
    #''')
    
    robjects.r('''
        library(dorothea)
        subset_dth = dorothea_{0}[dorothea_{0}$confidence %in% c({1}), ]
        '''.format('hs' if organism=='human' else 'mm', confidenceR))
    robjects.r('''
        df = data.frame(source = subset_dth$tf,
                        sign = subset_dth$mor,
                        target = subset_dth$target)
        write.table(df, file="{0}dorothea{2}_{3}_{1}.sif", sep = "\t", col.names = FALSE, row.names = FALSE, quote = FALSE)
        '''.format(directory_output, date.strftime("%Y%m%d"), confidence, organism))

In [9]:
dorothea_extraction(organism="mouse", confidence="ABC")

### 4. Standardize the interaction graph gene names.

In [ ]:
import os
from typing import List, Set, Dict, Tuple


def get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata: str) -> Dict:
    """
    Create a dictionary matching each possible gene name to its NCBI symbol.
    
    Particularity:
    Creation of a temporary file for speeding up the task facing a large matrix from NCBI, the parsing of the NCBI gene data is run with awk. A temporary file is then created.
    
    INPUT
        path_NCBIgenedata: path to the NCBI gene data
    OUTPUT
        dictionary (key: gene name, value: reference gene name (being the NCBI symbol))
    """
    
    # Parse the downloaded NCBI gene data:
    path_NCBIgenedata_cut = f"{path_NCBIgenedata}_cut"
    command_parsing = "awk -F'\t' '{print $3 \"\t\" $5 \"\t\" $11}' " + path_NCBIgenedata + " | tr \| '\t' > " + path_NCBIgenedata_cut + " ; sed -i 1d " + path_NCBIgenedata_cut
    os.system(command_parsing)
    
    # Extract gene data information:    
    gene_synonyms_dict = dict()
    symbols = set()

    with open (path_NCBIgenedata_cut, "r") as file_synonyms:
        for gene in file_synonyms:
            gene = gene.strip().upper()
            gene_symbols_list = gene.split("\t")
            #extract reference gene symbol:
            ncbi_symbol = gene_symbols_list.pop(0)
            #delete non-informative synonyms:
            res = [syn for syn in gene_symbols_list if (syn != "-" and syn != ncbi_symbol)]

            #create the dictionnary matching each symbol to its reference gene symbol:
            gene_synonyms_dict[ncbi_symbol] = ncbi_symbol
            symbols.add(ncbi_symbol)

            for gene in res:
                if gene not in symbols:
                    # Warning with NCBI list of synonyms:
                    # A noun can be the synonym of several symbols.
                    # Arbitrary, the choosen one is the first.
                    gene_synonyms_dict[gene] = ncbi_symbol
                    
    os.system(f"rm {path_NCBIgenedata_cut}")
    return gene_synonyms_dict


def get_reference_gene_name(gene_name: str, dict_synonyms: dict) -> str:
    """
    Given a gene name, return its reference name.
    INPUT
        dict_synonyms
        gene_name: the gene name you want its reference name
    OUTPUT
        the synonym considered as the reference name
    """
    gene_name = gene_name.upper()
    if gene_name in dict_synonyms:
        return dict_synonyms[gene_name]
    return gene_name

In [ ]:
def standardize_genename_in_file(path_input: str, path_NCBIgenedata: str, columns_to_standardize: List or Set[str] = [0], sep = "\t"):
    """
    Create a copy of the input file, with each gene name replaced by its reference (NCBI symbol) in the column precised in argument.
    
    Require the following functions:
        get_dict_matching_synonyms_to_refgenename
        get_reference_gene_name
       
    INPUT
        path_input: path to the input file in which the names must be standardized.
        path_NCBIgenedata: path to the NCBI gene data.
        columns_to_standardize : the columns containing gene names we want to standardize. Columns must start at index 0.
        sep: the field separator into the input SIF file (the gene data file provided by NCBI is a TSV).
    OUTPUT
        copy of the input file but with each gene into the columns_to_standardize named by its reference gene name (capitalized NCBI symbol). Named as the input file with at its end the extension "_reference-gene-names".
    """
    
    # Get gene data information:
    gene_synonyms_dict = get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata)
    
    # Replace gene name with reference gene name into the columns_to_standardize of the input file:
    cols_check = set() #put all elements of columns_to_standardize in a set for complexity 
    for c in columns_to_standardize:
        cols_check.add(c)
    
    with open(path_input, "r") as inputfile:
        to_write = []
        for ligne in inputfile.read().split("\n"):
            if len(ligne) > 1:
                cols = ligne.split(sep)
                ligne_output = ""
                id_col = 0
                for col in cols[:-1]:
                    if id_col in cols_check:
                        ligne_output += get_reference_gene_name(col, gene_synonyms_dict) + sep
                    else:
                        ligne_output += col + sep
                    id_col += 1
                if id_col in cols_check:
                    ligne_output += get_reference_gene_name(cols[-1], gene_synonyms_dict)
                else:
                    ligne_output += cols[-1]
                ligne_output += "\n"
                to_write.append(ligne_output)
    
    with open(f"{path_input}_reference-gene-names", "x") as outputfile:
        for ligne in to_write:
            outputfile.write(ligne)

In [ ]:
# Copy of 2022-10-06_dorotheaABC_mouse.sif by replacing each genename with its NCBI symbol
standardize_genename_in_file("dorotheaABC_mouse_20221010.sif", "Mus_musculus.gene_info.20221005.tsv", (0,2), "\t")

### 5. Eliminate genes with both no influence on others into the interaction graph and no expression value into the single-cell data.

In [ ]:
sources = set()  #stores the sources of edges
targets = set()  #stored the targets of edges
with open("dorotheaABC_mouse_20221010.sif_reference-gene-names", "r") as f:
    for edge in f:
        edge = edge.strip().split()
        sources.add(edge[0])
        targets.add(edge[2])

In [ ]:
tf_tf_interactions = sources.intersection(targets)  #Which targets are also sources?

print(f"TF 'source' which are also 'targets': {len(tf_tf_interactions)}\n\
Number of TF with expression value in the single-cell dataset: {len(sources.intersection(standardized_genenames_in_singlecell))}\n\
Number of TF 'targets' with expression value in the single-cell dataset: {len(tf_tf_interactions.intersection(standardized_genenames_in_singlecell))}")

In [ ]:
with open("dorotheaABC_mouse_20221010.sif_reference-gene-names", "r") as f_in:
    with open("standardized_dorothea_20221010.sif", "w") as f_out:
        # In path_fichier_noms_NCBI, a line = an edge.
        # A line is written in the output file if:
        # the edge source is a target of at least one edge
        # the edge target is a source of at least one edge or belongs to the Nestorowa list of genes
        for line in f_in:
            edge = line.strip().split()
            if edge[2] in tf_tf_interactions or edge[2] in standardized_genenames_in_singlecell:
                f_out.write(line)

In [ ]:
import networkx as nx

df = pd.read_csv("standardized_dorothea_20221010.sif", header=None, names=("in", "sign", "out"), sep="\t")
tf_tf_nesto = nx.from_pandas_edgelist(df, "in", "out", ["sign"], nx.MultiDiGraph())

print(f"TF→TF & TF→Nesto graph: {len(tf_tf_nesto.nodes())} nodes, {len(tf_tf_nesto.edges())} edges\n\
{len(set(tf_tf_nesto).intersection(standardized_genenames_in_singlecell))} nodes observed in Nestorowa dataset")

In [ ]:
import os

os.system("rm dorotheaABC_mouse_20221010.sif dorotheaABC_mouse_20221010.sif_reference-gene-names")

## BoNesis selection of components

In [10]:
import bonesis

In [11]:
standardized_dorothea = bonesis.InfluenceGraph.from_sif("standardized_dorothea_20221010.sif", maxclause=8, allow_skipping_nodes=True, canonic=False)

In [12]:
bo = bonesis.BoNesis(standardized_dorothea, standardized_observations_from_singlecell_nestorowa)

In [21]:
print(f"domain: {len(standardized_dorothea.nodes())} nodes, {len(standardized_dorothea.edges())} edges")

domain: 1198 nodes, 3550 edges


In [19]:
print(f"observations:")
for k,v in sorted(standardized_observations_from_singlecell_nestorowa.items()):
    print(f"{k}: {len(standardized_observations_from_singlecell_nestorowa[k])} genes with an expression value")

observations:
S0: 2330 genes with an expression value
S1: 2415 genes with an expression value
S2: 2476 genes with an expression value
S3: 2449 genes with an expression value
S4: 2435 genes with an expression value
S5: 2362 genes with an expression value


### Dynamics
<img src="trajectoire.png" alt="nestorowa stream trajectory" style="width:40%;"/>

#### fixpoints

In [22]:
s2 = bo.fixed(~bo.obs("S2"))
s4 = bo.fixed(~bo.obs("S4"))
s5 = bo.fixed(~bo.obs("S5"))
s2 != s4
s5 != s4
s2 != s5;

#### positive reachability

In [23]:
~bo.obs('S1') >= ~bo.obs('S0') >= s2
~bo.obs('S0') >= ~bo.obs('S3') >= s4
~bo.obs('S3') >= s5;

#### negative reachability

In [24]:
~bo.obs("S3") / s2;

### Optimization & view

In [26]:
bo.maximize_nodes()
bo.maximize_strong_constants()

<bonesis.language.ManagedIface.__init__.<locals>.managed.<locals>.Managed at 0x7f3dbe899bb0>

In [28]:
view = bonesis.NonStrongConstantNodesView(bo, mode="opt")

In [29]:
view.standalone(output_filename=f"maxnodes_maxstrongconstant.sh")